In [1]:
# Login to Hugging Face
from huggingface_hub import login

# Option 1: Will prompt for token
# login()

# Option 2: Use token directly (replace with your token)
# login(token="YOUR_HF_TOKEN_HERE")

# Option 3: Read from environment variable
# import os
# login(token=os.environ.get("HF_TOKEN"))


/mnt/task_runtime/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-Math-7B-Instruct")

In [26]:
# Load model directly

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-Math-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

#outputs = model.generate(**inputs, max_new_tokens=40)
#print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.76it/s]


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [13]:
from datasets import load_dataset

ds = load_dataset("KbsdJames/Omni-MATH")

ds

DatasetDict({
    test: Dataset({
        features: ['domain', 'difficulty', 'problem', 'solution', 'answer', 'source'],
        num_rows: 4428
    })
})

In [ ]:
# Load the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM

#finetuned_tokenizer = AutoTokenizer.from_pretrained("./qwen_finetuned")
finetuned_model = AutoModelForCausalLM.from_pretrained("./models/qwen2-math-7b-instruct_grpo_omni_math")
finetuned_model.to(device)


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.00it/s]
The module name qwen2_hyphen_math_hyphen_7b_hyphen_instruct_finetuned_on_first_3542_transformed_omni_math_solutions_filtered_lr:2e_hyphen_06_warmup_steps:300_num_epochs:3 (originally qwen2-math-7b-instruct_finetuned_on_first_3542_transformed_omni_math_solutions_filtered_lr:2e-06_warmup_steps:300_num_epochs:3) is not a valid Python identifier. Please rename the original module to avoid import issues.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,)

In [115]:
# Get a geometry level 5 problem from the dataset
#geometry_level5 = ds['train'].filter(lambda x: x.get('type') == 'Algebra' and x.get('level') == 'Level 4')
#problem = geometry_level5[11] #9
model.to(device)
num_problems = len(ds['test'])
problem = ds['test'].select(range(int(num_problems*0.8) + 1, len(ds['test'])))[10]
print(problem)

# Format the problem for the model
messages = [
    #{"role": "user", "content": f"Solve this geometry problem without using any external tools. Put your solution in \\boxed{...} format.\n\n Here is the problem:\n\n{problem['problem']}"},
    {"role": "system", "content": """You are a math tutor. Give a complete solution using the environments \\begin{intermediatederivation}...\\end{intermediatederivation} and \\begin{lemmatheorembox}...\\end{lemmatheorembox}, and put the final answer in the format \\boxed{...} at the end."""},
    #{"role": "system", "content": """You are a math tutor. Give a complete solution and put the final answer in the format \\boxed{...}"""}, 
    {"role": "user", "content": f"""{problem['problem']}"""}
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(device)

inputs


{'domain': ['Mathematics -> Number Theory -> Prime Numbers', 'Mathematics -> Number Theory -> Congruences'], 'difficulty': 6.5, 'problem': 'Find all primes $p$ and $q$ such that $3p^{q-1}+1$ divides $11^p+17^p$', 'solution': "\nTo solve the problem, we need to identify all pairs of primes \\( p \\) and \\( q \\) such that the expression \\( 3p^{q-1} + 1 \\) divides \\( 11^p + 17^p \\). The reference answer indicates that the only solution is the pair \\((3, 3)\\). Let's go through the process of verifying this.\n\nGiven the division condition:\n\n\\[ \n3p^{q-1} + 1 \\mid 11^p + 17^p\n\\]\n\nwe need to explore the values of \\( p \\) and \\( q \\). First, consider small prime numbers for both \\( p \\) and \\( q \\) due to the computational feasibility and complexity considerations.\n\n### Case: \\( p = 3 \\)\n\nWhen \\( p = 3 \\), evaluate \\( 3p^{q-1} + 1 \\) and \\( 11^p + 17^p \\):\n\n- For \\( p = 3 \\), the expression becomes \\( 3 \\times 3^{q-1} + 1 \\). This simplifies to \\( 3

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,    264,   6888,  25302,     13,
          20678,    264,   4583,   6291,   1667,    279,  21737,   1124,   7265,
             90,   2245,   4404,    657,    261,  39127,     92,  30801,    408,
             90,   2245,   4404,    657,    261,  39127,     92,    323,   1124,
           7265,     90,   3433,   8470,    383,  13173,   2011,     92,  30801,
            408,     90,   3433,   8470,    383,  13173,   2011,   2137,    323,
           2182,    279,   1590,   4226,    304,    279,   3561,   1124,  79075,
             90,   1112,     92,    518,    279,    835,     13, 151645,    198,
         151644,    872,    198,   9885,    678,  49433,    400,     79,      3,
            323,    400,     80,      3,   1741,    429,    400,     18,     79,
          47822,     80,     12,     16,     92,     10,     16,      3,  64828,
            400,     16,     16,     61,     79,     10,     16,     22,     61,
             7

In [117]:
outputs = finetuned_model.generate(**inputs, max_new_tokens=6144, temperature=0.7)
response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print("Problem:")
print(problem['problem'])
print("\n" + "="*80 + "\n")
print("Model Response:")
print(response)

Problem:
Find all primes $p$ and $q$ such that $3p^{q-1}+1$ divides $11^p+17^p$


Model Response:
\begin{intermediatederivation}
Consider the problem of finding all primes p and q such that $3^{p-1}+1$ divides $11^p+17^p$. One approach is to use the Lifting The Exponent (LTE) lemma. The LTE lemma states that for odd prime p, if $a \equiv b \pmod{p}$ and $a \not \equiv b \pmod{p^2}$, then $v_p(a^n - b^n) = v_p(a-b)+v_p(n)$. We can apply this lemma to the given problem.
\end{intermediatederivation}

\begin{lemmatheorembox}
\textbf{Name:} Lifting The Exponent Lemma
\textbf{Statement:} Let p be an odd prime and let a and b be positive integers such that $p \nmid a$, $p \nmid b$, and $p \mid a-b$. Then $v_p(a^n - b^n) = v_p(a-b)+v_p(n)$ for any positive integer n.
\end{lemmatheorembox}

\begin{intermediatederivation}
Using the LTE lemma, we have $v_3(11^p + 17^p) = v_3(11 + 17) = v_3(28) = 1$ and $v_3(3^{p-1} + 1) = v_3(3^2 + 1) = v_3(10) = 1$. Therefore, we have $v_3(11^p + 17^p) = v_3(3^{

In [34]:
problem['solution']

'When we rotate images $90^{\\circ}$ the coordinates switch places, and the signs are adjusted based on whether or not an axis was crossed. In this case, rotating point $A$ $90^{\\circ}$ will bring it across the $y$-axis into Quadrant I, which means both the $x$ and $y$ will be positive. The original point $A$ was at $(-4, 1)$ so the final image will be at $(1, 4)$. We also could solve this problem by seeing that the slope of the segment from the origin to $A$ is $-1/4$. If $A$ is moving to a location that is a $90^{\\circ}$ rotation about the origin, it will move to a point on the segment perpendicular to the one that currently connects it to the origin. This will be the segment that has a slope of 4/1 or $-4/-1$ from the origin which puts us at $(1, 4)$ or $(-1, -4)$. The point $\\boxed{(1, 4)}$ is in the clockwise direction we need.'

In [26]:
import datasets
generated_dataset_level5 = datasets.load_from_disk('data/math_solutions_dataset_20000/')

In [ ]:
generated_dataset_level5

[0]

In [15]:
# Reload the dataset to ensure it's loaded correctly
from datasets import load_from_disk

generated_dataset_level5 = load_from_disk('data/math_solutions_dataset_20000/')

# Check the dataset structure
print(f"Dataset type: {type(generated_dataset_level5)}")
print(f"Dataset length: {len(generated_dataset_level5)}")
if hasattr(generated_dataset_level5, 'features'):
    print(f"Dataset features: {generated_dataset_level5.features}")
    print(f"\nFirst example:")
    print(generated_dataset_level5[0])
else:
    print(f"Error: Dataset is not a proper Dataset object. It's a {type(generated_dataset_level5)}")
    print(f"First item: {generated_dataset_level5[0] if len(generated_dataset_level5) > 0 else 'empty'}")


Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Dataset length: 553
Dataset features: {'problem': Value('string'), 'ground_truth': Value('string'), 'solution': Value('string')}

First example:
{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.', 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so 

In [ ]:
generated_dataset_level5[0]

{'problem': 'Square ABCD has its center at $(8,-8)$ and has an area of 4 square units. The top side of the square is horizontal. The square is then dilated with the dilation center at (0,0) and a scale factor of 2. What are the coordinates of the vertex of the image of square ABCD that is farthest from the origin? Give your answer as an ordered pair.',
 'ground_truth': 'With the center of dilation at the origin and a scale factor of 2, all the coordinates of square $ABCD$ are twice the coordinates of its preimage. The preimage has an area of 4 square units, so its side length is 2 units. Since the center of the preimage is at $(8, -8)$, the four vertices of the preimage are at $(7, -9), (7, -7), (9, -7)$ and $(9, -9)$. The point $(9, -9)$ is the farthest from the origin on the preimage, so the point farthest from the origin on the image of square $ABCD$ is $\\boxed{(18, -18)}.$',
 'solution': "\\begin{intermediatederivation}\nThe square has area \\(4\\), so its side length is \\(s=\\sq

In [1]:
import datasets
dataset = datasets.load_dataset('KbsdJames/Omni-MATH')
#filtered_dataset = datasets.load_from_disk('newopenaioutputs/transformed_solutions_qwen2-math-7b-instruct_filtered')

/mnt/task_runtime/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    test: Dataset({
        features: ['domain', 'difficulty', 'problem', 'solution', 'answer', 'source'],
        num_rows: 4428
    })
})